In [2]:
from IPython.display import display, Image, Audio

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv('.env')

# Now you can access the environment variables
api_key = os.getenv('OPEN_AI_KEY')

# 

In [10]:
client = OpenAI(api_key = api_key)


In [3]:
import cv2
import numpy as np

# Load the YOLO model
net = cv2.dnn.readNetFromDarknet('/Users/sameet/Crime GPT/yolov3/yolov3.cfg', '/Users/sameet/Crime GPT/yolov3/yolov3.weights')

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Open the video file
video = cv2.VideoCapture("/Users/sameet/Crime GPT/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4")

# Flag to indicate if a person is detected
person_detected = False

# Loop through the frames of the video
while True:
    # Read the frame
    ret, frame = video.read()
    if not ret or person_detected:
        break
    
    # Resize the frame
    frame = cv2.resize(frame, None, fx=0.4, fy=0.4)
    
    # Convert the frame to a blob
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
    
    # Set the input to the network
    net.setInput(blob)
    
    # Forward pass through the network
    outs = net.forward(output_layers)
    
    # Process the outputs
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Person class
                person_detected = True  # Set the flag to True if a person is detected
                # Get the coordinates of the bounding box
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                
                # Add the bounding box information to the lists
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                break  # Exit the loop as a person is detected
    
    if person_detected:
        break  # Exit the main loop as a person is detected
    
    # Apply non-maximum suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Draw the bounding boxes on the frame
    for i in indices:
        box = boxes[i]
        left, top, width, height = box
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Person Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and close the windows
video.release()
cv2.destroyAllWindows()

# Return the flag
print(f"Person detected: {person_detected}")


Person detected: True


In [11]:
import cv2

# Open the video file
video = cv2.VideoCapture("/Users/sameet/Crime GPT/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4")

# Set the start and end time for trimming
start_time = 0  # Start time in seconds
end_time = 30  # End time in seconds

# Convert time to frame indices
fps = video.get(cv2.CAP_PROP_FPS)
start_frame = int(start_time * fps)
end_frame = int(end_time * fps)

# Read and discard frames until the start frame
for _ in range(start_frame):
    success, _ = video.read()
    if not success:
        break

# Read and process frames within the desired range
base64Frames = []
for _ in range(start_frame, end_frame):
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")



PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Categorize this video in crime or not crime. If it is a crime generate a description",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)


900 frames read.
Based on the frames provided, it appears the video depicts a physical altercation involving multiple individuals. This type of scene can be categorized under crime, specifically assault.

### Description:

A violent confrontation took place in what appears to be a public area. Multiple individuals are seen involved in a physical fight, with one or more subjects engaged in striking and kicking others. A warning screen at the beginning of the video indicates that the footage may be upsetting to some viewers. This incident could potentially be classified as assault and may be subject to legal investigation depending on the jurisdiction.


In [25]:
# video = cv2.VideoCapture("/Users/sameet/Crime GPT/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4")

# base64Frames = []
# while video.isOpened():
#     success, frame = video.read()
#     if not success:
#         break
#     _, buffer = cv2.imencode(".jpg", frame)
#     base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# video.release()
# print(len(base64Frames), "frames read.")

2486 frames read.


In [ ]:
display_handle = display(None, display_id=True)
for img in base64Frames:
    display_handle.update(Image(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.025)

In [36]:


PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Categorize this video in crime or not crime. If it is a crime generate a description",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

Based on the frames provided, the video appears to capture a violent altercation among several individuals. Given that physical violence is portrayed, this categorizes the video as involving a crime.

### Description:
The video depicts a violent street altercation involving multiple individuals. The footage shows various phases of the incident, where the individuals are seen engaging in physical combat. One person appears to be beaten by a group, sustaining blows while on the ground. The situation seems intense and potentially harmful, emphasizing the severity of the aggression displayed.

Given the nature content, if you decide to upload, ensure you include appropriate warnings and consider the platform’s guidelines to avoid content removal or violations of community standards.


: 

In [1]:
import cv2
import numpy as np
import base64
import openai
from IPython.display import display, Image, Audio
import time
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('.env')

api_key = os.getenv('OPEN_AI_KEY')
client = OpenAI(api_key=api_key)

net = cv2.dnn.readNetFromDarknet('/Users/sameet/Crime GPT/yolov3/yolov3.cfg', '/Users/sameet/Crime GPT/yolov3/yolov3.weights')

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

video = cv2.VideoCapture("/Users/sameet/Crime GPT/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4")

person_detected = False

while True:
    
    ret, frame = video.read()
    if not ret or person_detected:
        break
    
    frame = cv2.resize(frame, None, fx=0.4, fy=0.4)
    
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
    
    net.setInput(blob)
    
    outs = net.forward(output_layers)
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  
                person_detected = True  
                break  
        if person_detected:
            break 

    if person_detected:
        break  
if person_detected:
    # Reopen the video file to process frames from the beginning
    video = cv2.VideoCapture("/Users/sameet/Crime GPT/Anomaly-Videos-Part-1/Assault/Assault001_x264.mp4")

    start_time = 0  
    end_time = 30  

    fps = video.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    for _ in range(start_frame):
        success, _ = video.read()
        if not success:
            break

    base64Frames = []
    for _ in range(start_frame, end_frame):
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

    video.release()
    print(len(base64Frames), "frames read.")

    PROMPT_MESSAGES = [
        {
            "role": "user",
            "content": [
                "These are frames from a video that I want to upload. Categorize this video in crime or not crime. If it is a crime generate a description",
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
            ],
        },
    ]
    params = {
        "model": "gpt-4o",
        "messages": PROMPT_MESSAGES,
        "max_tokens": 200,
    }

    result = client.chat.completions.create(**params)
    print(result.choices[0].message.content)

video.release()
cv2.destroyAllWindows()


900 frames read.
The video frames depict a violent altercation involving multiple individuals. The sequence shows one or more individuals assaulting another person who appears to be on the ground at some points. Given the nature of the actions displayed in the frames, this situation can be classified as a potential crime, particularly physical assault.

### Description:
The video captures a street brawl involving multiple assailants attacking an individual. The footage shows one or more people repeatedly striking the victim while they are on the ground, suggesting a violent physical assault. The scenario unfolds rapidly and appears to be unprovoked. It portrays a clear instance of aggression and physical violence, likely resulting in harm to the targeted individual. 

Viewer discretion is advised due to the graphic nature of the content.


In [3]:
!pip install flask opencv-python-headless openai python-dotenv




Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 59.1 MB/s eta 0:00:00a 0:00:01
